# Notebook to get Summary and Subject Using AI Integration

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
# import scispacy
import re
import tqdm
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from bertopic import BERTopic
from openai import OpenAI
import hdbscan
import os
from dotenv import load_dotenv
import ast
import sqlite3

/Users/vanikanoria/anaconda3/envs/topic-modeling/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Get OpenAI key (saved as environment variable)

In [3]:
load_dotenv()

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"))

### Summarization using LLMs

In [4]:
messages = pd.read_csv('messages.csv')  # Load your messages dataframe


In [5]:
msg = messages.iloc[1]['content']  # Example message for testing
msg

"1. What is your job title/department? Animal Care Specialist at Riverside County Animal Services.\n2. Who is the department supervisor? Robert Brown\n3. Brief description of accident: As I was exiting the dog's kennel he got excited and and started jumping up and biting at me.\n4. Size of the dog and location on body that you were bitten: Large sized dog and under my right upper arm.\n5. Prior injury to the affected body part? No\n6. Any medical conditions that require maintenance medication? No\n7. Have you missed any time from work as a result of this accident? Just 2 hours on the date of the accident (6/3/24) because I left earlier to see a doctor.\n\nThanks!\n-Sarah Cooper"

In [6]:
SYSTEM = "You extract a short SUBJECT (≤5 words) and a 1 sentence SUMMARY for insurance claim messages. Return JSON."
user = f"Message:\n\"\"\"\n{msg}\n\"\"\"\nRespond as: {{\"subject\":\"...\",\"summary\":\"...\"}}"


In [7]:
resp = client.chat.completions.create(
    model="gpt-4o-mini",  # or your available model
    messages=[{"role":"system","content":SYSTEM},{"role":"user","content":user}],
    # avoid temperature if your model complains; otherwise use 0–0.2 for stability
    max_tokens=120
)
data = json.loads(resp.choices[0].message.content)

In [8]:
data

{'subject': 'Dog Bite Incident',
 'summary': 'Sarah Cooper, an Animal Care Specialist, was bitten by a large dog on her right upper arm while exiting its kennel, resulting in a 2-hour absence from work to seek medical attention.'}

Function to get summary and subject

In [ ]:
def summarize_and_extract_subject_and_summary2(conversation, model="gpt-4o-mini"):
    """
    Summarizes a conversation and extracts a short subject line.
    Returns the raw AI model output instead of parsing subject/summary.

    Args:
        conversation (str): Full text of the conversation or message thread.
        model (str): OpenAI model to use.

    Returns:
        str: The raw AI response text.
    """
    prompt = f"""
    You are given a conversation between a claimant and adjuster.
    1. Provide a short subject line (max 8 words) summarizing the main topic. 
    2. Provide a concise summary (1 sentence) covering the key points.

    Provide in JSON format:
    {{
        "subject": "<short subject line>",
        "summary": "<concise summary>"
    }}

    Conversation:
    {conversation}
    """

    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )

    # Return raw model output directly
    return completion.choices[0].message.content.strip()

In [26]:
# Example usage:
conversation_text = """
Claimant: I sent the reimbursement form last week. Have you received it?
Adjuster: Yes, but it’s missing the mileage section.
Claimant: Okay, I’ll resend with the missing info.
"""
result = summarize_and_extract_subject_and_summary(conversation_text)
print(result)

{'subject': 'Reimbursement Form Missing Information', 'summary': 'The claimant needs to resend the reimbursement form due to a missing mileage section.'}


In [11]:
from tqdm import tqdm
tqdm.pandas()

### Function to extract subject and summary from JSON

## Apply to dataset

In [12]:
whole_output = messages['content'].progress_apply(
    lambda x: summarize_and_extract_subject_and_summary2(x, model="gpt-4o-mini")
)

  0%|          | 0/3079 [00:00<?, ?it/s]

100%|██████████| 3079/3079 [55:22<00:00,  1.08s/it]  


In [13]:
messages['whole_output'] = whole_output

# Save to CSV
output_path = "messages_with_whole_output.csv"
messages.to_csv(output_path, index=False)

print(f"Saved updated messages to {output_path}")

Saved updated messages to messages_with_whole_output.csv


In [14]:
messages['whole_output']

0       ```json\n{\n    "subject": "Discussion on Acci...
1       ```json\n{\n    "subject": "Dog Bite Incident ...
2       {\n    "subject": "Thank You from Claims Super...
3       {\n    "subject": "Follow-Up Appointment Confi...
4       {\n    "subject": "Appointment Rescheduling Up...
                              ...                        
3074    {\n    "subject": "Claimant Declines Offer",\n...
3075    ```json\n{\n    "subject": "Medical Reaction P...
3076    {\n    "subject": "Paperwork Submission Confir...
3077    {\n    "subject": "Return to Work Update",\n  ...
3078    {\n    "subject": "Claim Acknowledgment",\n   ...
Name: whole_output, Length: 3079, dtype: object

In [17]:
def extract_subject_summary_from_str(val):
    """
    Extracts subject and summary from a JSON-like string or dict.
    Handles cases with ```json code fences and escaped newlines.
    """
    if pd.isna(val):
        return "", ""

    # If it's already a dict
    if isinstance(val, dict):
        return val.get("subject", ""), val.get("summary", "")

    # Ensure it's a string
    s = str(val).strip()

    # Remove code block markers like ```json ... ```
    s = re.sub(r"^```(?:json)?", "", s.strip(), flags=re.IGNORECASE)
    s = re.sub(r"```$", "", s.strip())

    # Remove leading/trailing quotes if present
    s = s.strip('"').strip("'")

    # Try JSON first
    try:
        parsed = json.loads(s)
        if isinstance(parsed, dict):
            return parsed.get("subject", ""), parsed.get("summary", "")
    except json.JSONDecodeError:
        pass

    # Try literal_eval (for Python dict string format)
    try:
        parsed = ast.literal_eval(s)
        if isinstance(parsed, dict):
            return parsed.get("subject", ""), parsed.get("summary", "")
    except Exception:
        pass

    # If parsing fails
    return "", ""

# Example usage:
messages[["subject", "summary"]] = messages["whole_output"].apply(
    lambda x: pd.Series(extract_subject_summary_from_str(x))
)

In [21]:
messages[["subject", "summary"]].head(40)

,subject,summary
0,Discussion on Accident Details,The adjuster requests information from Ms. Coo...
1,Dog Bite Incident Report,Claimant Sarah Cooper reports a dog bite incid...
2,Thank You from Claims Supervisor,Emily Martinez expresses gratitude to the clai...
3,Follow-Up Appointment Confirmation,The adjuster is inquiring if the claimant atte...
4,Appointment Rescheduling Update,The claimant rescheduled their missed appointm...
5,Request for Rescheduled Appointment Paperwork,The claimant is requesting paperwork from a re...
6,Voicemail Acknowledgment,The claimant acknowledges receiving the adjust...
7,Missing Paperwork from MediFirst Care,The claimant is requesting paperwork from Medi...
8,Claimant's Confirmation,The claimant confirms their ability to proceed.
9,Gratitude Expressed,The claimant expresses appreciation to the adj...


In [23]:
messages = messages.reset_index().rename(columns={'index': 'message_id'})

In [24]:
messages

,message_id,thread_id,timestamp,role,content,whole_output,subject,summary
0,0,0,2024-06-07 14:47:59,supervisor,"Good morning, Ms. Cooper - when you have a few...","```json\n{\n ""subject"": ""Discussion on Acci...",Discussion on Accident Details,The adjuster requests information from Ms. Coo...
1,1,0,2024-06-10 13:18:38,claimant,1. What is your job title/department? Animal C...,"```json\n{\n ""subject"": ""Dog Bite Incident ...",Dog Bite Incident Report,Claimant Sarah Cooper reports a dog bite incid...
2,2,0,2024-06-10 13:39:40,supervisor,Thank you!!\n\nEmily Martinez\nSupervisor WC T...,"{\n ""subject"": ""Thank You from Claims Super...",Thank You from Claims Supervisor,Emily Martinez expresses gratitude to the clai...
3,3,0,2024-07-01 19:03:40,supervisor,Hi Sarah - According to the documents from you...,"{\n ""subject"": ""Follow-Up Appointment Confi...",Follow-Up Appointment Confirmation,The adjuster is inquiring if the claimant atte...
4,4,0,2024-07-02 3:35:15,claimant,I was unable to make it to an appointment on 6...,"{\n ""subject"": ""Appointment Rescheduling Up...",Appointment Rescheduling Update,The claimant rescheduled their missed appointm...
...,...,...,...,...,...,...,...,...
3074,3074,946,2022-07-01 13:39:39,claimant,No thanks,"{\n ""subject"": ""Claimant Declines Offer"",\n...",Claimant Declines Offer,The claimant has chosen to decline the offer p...
3075,3075,947,2022-10-04 13:30:59,claimant,"Sarah, if you recall in the documentation ther...","```json\n{\n ""subject"": ""Medical Reaction P...",Medical Reaction Post-Accident,The claimant discusses an allergic reaction an...
3076,3076,948,2024-09-05 13:56:09,claimant,"Hello, Ms.Smith I sent you & sign the paperwor...","{\n ""subject"": ""Paperwork Submission Confir...",Paperwork Submission Confirmation,Ms. Smith confirms sending and signing the pap...
3077,3077,948,2024-09-13 12:32:02,claimant,"Ms. Smith, I went back to work on September 3r...","{\n ""subject"": ""Return to Work Update"",\n ...",Return to Work Update,"Ms. Smith returned to work on September 3rd, 2..."


## Saving to avoid running the above function again

In [25]:
messages.to_csv('all_messages_with_subject_and_summary.csv')

In [26]:
messages = pd.read_csv('all_messages_with_subject_and_summary.csv')  # Load your messages dataframe

## Insert into Database

In [3]:
messages = pd.read_csv('all_messages_with_subject_and_summary.csv')  # Load your messages dataframe

In [4]:
## Look at current columns
def list_columns(db_path, table_name):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table_name})")
    columns = cur.fetchall()
    conn.close()
    return columns

cols = list_columns("claims.db", "messages")
for col in cols:
    print(col)

(0, 'message_id', 'TEXT', 0, None, 1)
(1, 'thread_id', 'TEXT', 0, None, 0)
(2, 'ts_iso', 'TEXT', 1, None, 0)
(3, 'ts_unix', 'INTEGER', 1, None, 0)
(4, 'role', 'TEXT', 0, None, 0)
(5, 'content', 'TEXT', 0, None, 0)
(6, 'intents_json', 'TEXT', 0, None, 0)
(7, 'intent_primary', 'TEXT', 0, None, 0)
(8, 'sentiment', 'REAL', 0, None, 0)
(9, 'language', 'TEXT', 0, None, 0)
(10, 'metadata_json', 'TEXT', 0, None, 0)
(11, 'source_file', 'TEXT', 0, None, 0)
(12, 'line_no', 'INTEGER', 0, None, 0)
(13, 'subject', 'TEXT', 0, None, 0)
(14, 'summary', 'TEXT', 0, None, 0)
(15, 'timestamp', 'TEXT', 0, None, 0)
(16, 'all_intents', 'TEXT', 0, None, 0)
(17, 'primary_intent', 'TEXT', 0, None, 0)
(18, 'primary_intents', 'TEXT', 0, None, 0)


In [ ]:
# # Connect to the database
# conn = sqlite3.connect("claims.db")
# cur = conn.cursor()

# # Run the UPDATE query
# cur.execute("""
#     UPDATE messages
#     SET message_id = substr(message_id, instr(message_id, ':') + 1)
#     WHERE instr(message_id, ':') > 0;
# """)

# # Commit and close
# conn.commit()
# conn.close()


## Update the columns to add subject and summary for matching rows


In [ ]:
## Add column titled subject
def add_column(db_path, table_name, column_name, column_type="TEXT"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    cur.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} {column_type}")
    conn.commit()
    conn.close()

# Example:
add_column("claims.db", "messages", "subject", "TEXT")

In [27]:

def ensure_column(db_path: str, table: str, column: str, coltype: str = "TEXT"):
    """Add a column if it doesn't exist."""
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()
    # check existing columns
    cols = [r[1] for r in cur.execute(f"PRAGMA table_info({table})").fetchall()]
    if column not in cols:
        cur.execute(f"ALTER TABLE {table} ADD COLUMN {column} {coltype}")
        conn.commit()
    conn.close()

def update_column_from_csv(
    csv_path: str,
    db_path: str,
    table_name: str,
    match_columns,  # can be str or list of str
    update_column: str,
    coltype: str = "TEXT",
    chunksize: int = 50_000,
):
    """
    Update `table_name.update_column` in SQLite from values in CSV,
    matching on one or more `match_columns`. CSV must contain all match columns and update_column.
    """

    # Normalize to list
    if isinstance(match_columns, str):
        match_columns = [match_columns]

    # 1) Ensure target column exists
    ensure_column(db_path, table_name, update_column, coltype)

    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    for chunk in pd.read_csv(csv_path, chunksize=chunksize):
        required_cols = set(match_columns + [update_column])
        if not required_cols.issubset(chunk.columns):
            raise ValueError(f"CSV must contain columns: {', '.join(required_cols)}")

        # Drop rows missing any of the required columns
        sub = chunk[list(required_cols)].dropna()

        # Build parameter tuples: (update_value, match_val1, match_val2, ...)
        params = [
            tuple([row[update_column]] + [row[col] for col in match_columns])
            for _, row in sub.iterrows()
        ]
        if not params:
            continue

        # SQL: UPDATE table SET col=? WHERE col1=? AND col2=? ...
        where_clause = " AND ".join([f"{col} = ?" for col in match_columns])
        sql = f"UPDATE {table_name} SET {update_column} = ? WHERE {where_clause}"

        cur.executemany(sql, params)
        conn.commit()

    conn.close()

    print(f"Updated {len(params)} rows for column {update_column}")


In [28]:
update_column_from_csv(
    csv_path="all_messages_with_subject_and_summary.csv",
    db_path="claims.db",
    table_name="messages",
    match_columns=["thread_id", "content"],         # or message_id, etc.
    update_column="subject",          # the column you’re adding/updating
    coltype="TEXT"
)

Updated 3078 rows for column subject


In [29]:
update_column_from_csv(
    csv_path="all_messages_with_subject_and_summary.csv",
    db_path="claims.db",
    table_name="messages",
    match_columns=["thread_id", "content"],       # or message_id, etc.
    update_column="summary",          # the column you’re adding/updating
    coltype="TEXT"
)

Updated 3078 rows for column summary
